In [18]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [17]:
##spark.stop()

In [19]:
spark

In [20]:
! hadoop fs -ls /public/trendytech/retail_db/order_items

Found 1 items
-rw-r--r--   3 itv005857 supergroup    5408880 2023-04-26 16:47 /public/trendytech/retail_db/order_items/part-00000


In [21]:
raw_df = spark.read.\
format("csv").\
option("inferSchema", "True").\
load("/public/trendytech/retail_db/order_items/part-00000")

In [22]:
raw_df.show()

+---+---+----+---+------+------+
|_c0|_c1| _c2|_c3|   _c4|   _c5|
+---+---+----+---+------+------+
|  1|  1| 957|  1|299.98|299.98|
|  2|  2|1073|  1|199.99|199.99|
|  3|  2| 502|  5| 250.0|  50.0|
|  4|  2| 403|  1|129.99|129.99|
|  5|  4| 897|  2| 49.98| 24.99|
|  6|  4| 365|  5|299.95| 59.99|
|  7|  4| 502|  3| 150.0|  50.0|
|  8|  4|1014|  4|199.92| 49.98|
|  9|  5| 957|  1|299.98|299.98|
| 10|  5| 365|  5|299.95| 59.99|
| 11|  5|1014|  2| 99.96| 49.98|
| 12|  5| 957|  1|299.98|299.98|
| 13|  5| 403|  1|129.99|129.99|
| 14|  7|1073|  1|199.99|199.99|
| 15|  7| 957|  1|299.98|299.98|
| 16|  7| 926|  5| 79.95| 15.99|
| 17|  8| 365|  3|179.97| 59.99|
| 18|  8| 365|  5|299.95| 59.99|
| 19|  8|1014|  4|199.92| 49.98|
| 20|  8| 502|  1|  50.0|  50.0|
+---+---+----+---+------+------+
only showing top 20 rows



In [23]:
raw_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)



In [24]:
refined_df = raw_df.toDF("order_item_id","order_id","product_id","quantity","subtotal","product_price")

In [25]:
refined_df.show()

+-------------+--------+----------+--------+--------+-------------+
|order_item_id|order_id|product_id|quantity|subtotal|product_price|
+-------------+--------+----------+--------+--------+-------------+
|            1|       1|       957|       1|  299.98|       299.98|
|            2|       2|      1073|       1|  199.99|       199.99|
|            3|       2|       502|       5|   250.0|         50.0|
|            4|       2|       403|       1|  129.99|       129.99|
|            5|       4|       897|       2|   49.98|        24.99|
|            6|       4|       365|       5|  299.95|        59.99|
|            7|       4|       502|       3|   150.0|         50.0|
|            8|       4|      1014|       4|  199.92|        49.98|
|            9|       5|       957|       1|  299.98|       299.98|
|           10|       5|       365|       5|  299.95|        59.99|
|           11|       5|      1014|       2|   99.96|        49.98|
|           12|       5|       957|       1|  29

In [26]:
df1= refined_df.drop("subtotal")

In [27]:
df1.show()

+-------------+--------+----------+--------+-------------+
|order_item_id|order_id|product_id|quantity|product_price|
+-------------+--------+----------+--------+-------------+
|            1|       1|       957|       1|       299.98|
|            2|       2|      1073|       1|       199.99|
|            3|       2|       502|       5|         50.0|
|            4|       2|       403|       1|       129.99|
|            5|       4|       897|       2|        24.99|
|            6|       4|       365|       5|        59.99|
|            7|       4|       502|       3|         50.0|
|            8|       4|      1014|       4|        49.98|
|            9|       5|       957|       1|       299.98|
|           10|       5|       365|       5|        59.99|
|           11|       5|      1014|       2|        49.98|
|           12|       5|       957|       1|       299.98|
|           13|       5|       403|       1|       129.99|
|           14|       7|      1073|       1|       199.9

In [28]:
df1.select("*").show()

+-------------+--------+----------+--------+-------------+
|order_item_id|order_id|product_id|quantity|product_price|
+-------------+--------+----------+--------+-------------+
|            1|       1|       957|       1|       299.98|
|            2|       2|      1073|       1|       199.99|
|            3|       2|       502|       5|         50.0|
|            4|       2|       403|       1|       129.99|
|            5|       4|       897|       2|        24.99|
|            6|       4|       365|       5|        59.99|
|            7|       4|       502|       3|         50.0|
|            8|       4|      1014|       4|        49.98|
|            9|       5|       957|       1|       299.98|
|           10|       5|       365|       5|        59.99|
|           11|       5|      1014|       2|        49.98|
|           12|       5|       957|       1|       299.98|
|           13|       5|       403|       1|       129.99|
|           14|       7|      1073|       1|       199.9

In [29]:
df1.select("order_item_id","order_id","product_id").show()

+-------------+--------+----------+
|order_item_id|order_id|product_id|
+-------------+--------+----------+
|            1|       1|       957|
|            2|       2|      1073|
|            3|       2|       502|
|            4|       2|       403|
|            5|       4|       897|
|            6|       4|       365|
|            7|       4|       502|
|            8|       4|      1014|
|            9|       5|       957|
|           10|       5|       365|
|           11|       5|      1014|
|           12|       5|       957|
|           13|       5|       403|
|           14|       7|      1073|
|           15|       7|       957|
|           16|       7|       926|
|           17|       8|       365|
|           18|       8|       365|
|           19|       8|      1014|
|           20|       8|       502|
+-------------+--------+----------+
only showing top 20 rows



In [30]:
from pyspark.sql.functions import expr

In [31]:
df1.select("*", expr("product_price * quantity as subtotal")).show()

+-------------+--------+----------+--------+-------------+--------+
|order_item_id|order_id|product_id|quantity|product_price|subtotal|
+-------------+--------+----------+--------+-------------+--------+
|            1|       1|       957|       1|       299.98|  299.98|
|            2|       2|      1073|       1|       199.99|  199.99|
|            3|       2|       502|       5|         50.0|   250.0|
|            4|       2|       403|       1|       129.99|  129.99|
|            5|       4|       897|       2|        24.99|   49.98|
|            6|       4|       365|       5|        59.99|  299.95|
|            7|       4|       502|       3|         50.0|   150.0|
|            8|       4|      1014|       4|        49.98|  199.92|
|            9|       5|       957|       1|       299.98|  299.98|
|           10|       5|       365|       5|        59.99|  299.95|
|           11|       5|      1014|       2|        49.98|   99.96|
|           12|       5|       957|       1|    

In [32]:
df1.selectExpr("*", "product_price * quantity as subtotal").show()

+-------------+--------+----------+--------+-------------+--------+
|order_item_id|order_id|product_id|quantity|product_price|subtotal|
+-------------+--------+----------+--------+-------------+--------+
|            1|       1|       957|       1|       299.98|  299.98|
|            2|       2|      1073|       1|       199.99|  199.99|
|            3|       2|       502|       5|         50.0|   250.0|
|            4|       2|       403|       1|       129.99|  129.99|
|            5|       4|       897|       2|        24.99|   49.98|
|            6|       4|       365|       5|        59.99|  299.95|
|            7|       4|       502|       3|         50.0|   150.0|
|            8|       4|      1014|       4|        49.98|  199.92|
|            9|       5|       957|       1|       299.98|  299.98|
|           10|       5|       365|       5|        59.99|  299.95|
|           11|       5|      1014|       2|        49.98|   99.96|
|           12|       5|       957|       1|    

In [33]:
product_raw_df = spark.read.\
format("csv").\
option("inferSchema", "True").\
load("/public/trendytech/retail_db/products/part-00000")

In [34]:
product_raw_df.show()

+---+---+--------------------+----+------+--------------------+
|_c0|_c1|                 _c2| _c3|   _c4|                 _c5|
+---+---+--------------------+----+------+--------------------+
|  1|  2|Quest Q64 10 FT. ...|null| 59.98|http://images.acm...|
|  2|  2|Under Armour Men'...|null|129.99|http://images.acm...|
|  3|  2|Under Armour Men'...|null| 89.99|http://images.acm...|
|  4|  2|Under Armour Men'...|null| 89.99|http://images.acm...|
|  5|  2|Riddell Youth Rev...|null|199.99|http://images.acm...|
|  6|  2|Jordan Men's VI R...|null|134.99|http://images.acm...|
|  7|  2|Schutt Youth Recr...|null| 99.99|http://images.acm...|
|  8|  2|Nike Men's Vapor ...|null|129.99|http://images.acm...|
|  9|  2|Nike Adult Vapor ...|null|  50.0|http://images.acm...|
| 10|  2|Under Armour Men'...|null|129.99|http://images.acm...|
| 11|  2|Fitness Gear 300 ...|null|209.99|http://images.acm...|
| 12|  2|Under Armour Men'...|null|139.99|http://images.acm...|
| 13|  2|Under Armour Men'...|null| 89.9

In [35]:
df1 = product_raw_df.toDF("product_id","product_category_id","product_name","product_description","product_price","product_image")

In [36]:
df1.show()

+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               null|        59.98|http://images.acm...|
|         2|                  2|Under Armour Men'...|               null|       129.99|http://images.acm...|
|         3|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         4|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               null|       199.99|http://images.acm...|
|         6|                  2|Jordan Men's VI R...|               null|       134.99|http://images.acm...|
|         7|       

In [37]:
df1.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_category_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_description: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_image: string (nullable = true)



In [38]:
df2 = df1.withColumn("product_price", expr("product_price*1.2"))

In [39]:
df2.show()

+----------+-------------------+--------------------+-------------------+------------------+--------------------+
|product_id|product_category_id|        product_name|product_description|     product_price|       product_image|
+----------+-------------------+--------------------+-------------------+------------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               null|            71.976|http://images.acm...|
|         2|                  2|Under Armour Men'...|               null|           155.988|http://images.acm...|
|         3|                  2|Under Armour Men'...|               null|107.98799999999999|http://images.acm...|
|         4|                  2|Under Armour Men'...|               null|107.98799999999999|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               null|           239.988|http://images.acm...|
|         6|                  2|Jordan Men's VI R...|               null|           161.

In [41]:
new_df = df1.withColumn("product_price", expr("CASE WHEN product_name like '%nike%' THEN product_price *1.2 WHEN product_name like '%Armour%' THEN product_price * 1.1  ELSE product_price END"))

In [42]:
new_df.show()

+----------+-------------------+--------------------+-------------------+------------------+--------------------+
|product_id|product_category_id|        product_name|product_description|     product_price|       product_image|
+----------+-------------------+--------------------+-------------------+------------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               null|             59.98|http://images.acm...|
|         2|                  2|Under Armour Men'...|               null|142.98900000000003|http://images.acm...|
|         3|                  2|Under Armour Men'...|               null|            98.989|http://images.acm...|
|         4|                  2|Under Armour Men'...|               null|            98.989|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               null|            199.99|http://images.acm...|
|         6|                  2|Jordan Men's VI R...|               null|            134

In [43]:
mylist = [(1,"kapil",34),(1,"kapil",34),(1,"satish",26),(2,"satish",26)]

In [44]:
df = spark.createDataFrame(mylist).toDF("id","name","age")

In [45]:
df.show()

+---+------+---+
| id|  name|age|
+---+------+---+
|  1| kapil| 34|
|  1| kapil| 34|
|  1|satish| 26|
|  2|satish| 26|
+---+------+---+



In [46]:
df1 = df.distinct().show()

+---+------+---+
| id|  name|age|
+---+------+---+
|  1|satish| 26|
|  2|satish| 26|
|  1| kapil| 34|
+---+------+---+



In [47]:
df1= df.select("id").distinct()

In [48]:
df1.show()

+---+
| id|
+---+
|  1|
|  2|
+---+



In [49]:
new_df = df.dropDuplicates()

In [50]:
new_df.show()

+---+------+---+
| id|  name|age|
+---+------+---+
|  1|satish| 26|
|  2|satish| 26|
|  1| kapil| 34|
+---+------+---+



In [51]:
df.dropDuplicates(["name","age"]).show()

+---+------+---+
| id|  name|age|
+---+------+---+
|  1| kapil| 34|
|  1|satish| 26|
+---+------+---+



In [52]:
df.dropDuplicates(["id"]).show()

+---+------+---+
| id|  name|age|
+---+------+---+
|  1|satish| 26|
|  2|satish| 26|
+---+------+---+



----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 51102)
Traceback (most recent call last):
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda3/envs/beakerx/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/opt/spark-3.1.2-bin-hadoop3.2/python/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/opt/spark-3.1.2-bin-hadoop3.2/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/opt/spark-3.1.2-bin-hadoop3.2/python/pyspark/accumulato